In [1]:
import h5py
import json
from tqdm import tqdm

categories = json.load(open('/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/rel_pruning/VG178/categories.json', 'r'))

vocab = "/home/maelic/Documents/SGDiff/datasets/vg/vocab.json"
vocab = json.load(open(vocab, 'r'))

def extract_rels(h5_file, categories, vocab):
    CLASSES = list(vocab['object_idx_to_name'])
    REL_CLASSES = list(vocab['pred_idx_to_name'])
    relations = []

    num_ood_rel = 0
    for i in tqdm(range(len(h5_file['image_ids']))):
        img_id = h5_file['image_ids'][i]
        img_path = h5_file['image_paths'][i]
        attributes_per_object = h5_file['attributes_per_object'][i]
        object_attributes = h5_file['object_attributes'][i]
        object_boxes = h5_file['object_boxes'][i]
        object_ids = h5_file['object_ids'][i]
        object_names = h5_file['object_names'][i]
        objects_per_image = h5_file['objects_per_image'][i]
        relationship_ids = h5_file['relationship_ids'][i]
        relationship_objects = h5_file['relationship_objects'][i]
        relationship_predicates = h5_file['relationship_predicates'][i]
        relationship_subjects = h5_file['relationship_subjects'][i]
        relationships_per_image = h5_file['relationships_per_image'][i]

        # get all objects
        objects = {}
        for j in range(objects_per_image):
            obj_id = object_ids[j]
            obj_name = CLASSES[object_names[j]]
            obj_box = object_boxes[j]
            obj_attributes = attributes_per_object[j]
            objects[j] = {"name": obj_name, "box": obj_box, "attributes": obj_attributes}

        # get all relations
        for j in range(relationships_per_image):
            rel_id = relationship_ids[j]
            rel_pred = REL_CLASSES[relationship_predicates[j]]
            rel_subj = objects[relationship_subjects[j]]['name']
            rel_obj = objects[relationship_objects[j]]['name']
            full_rel = str(rel_subj) + " " + str(rel_pred) + " " + str(rel_obj)
            if full_rel in categories:
                relations.append(categories[full_rel])
            else:
                num_ood_rel += 1

    print(num_ood_rel)
    unique, counts = np.unique(relations, return_counts=True)
    print(dict(zip(unique, counts)))

    return dict(zip(unique, counts))

h5_path = "/home/maelic/Documents/SGDiff/datasets/pred_transformers.h5"
h5_file = h5py.File(h5_path, 'r')

rels_count_1 = extract_rels(h5_file, categories, vocab)

h5_path = "/home/maelic/Documents/SGDiff/datasets/pred_transformers_intrinsic.h5"
h5_file = h5py.File(h5_path, 'r')

rels_count_2 = extract_rels(h5_file, categories, vocab)

100%|██████████| 4993/4993 [00:11<00:00, 445.31it/s]


350


NameError: name 'np' is not defined

In [ ]:
# display rels_count_1 and rels_count_2 as pie charts side by side using seaborn

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.DataFrame.from_dict(rels_count_1, orient='index', columns=['count'])
# transform count in relative percentage
df['count'] = df['count'] / df['count'].sum()
df['Model'] = 'Transformer'

df2 = pd.DataFrame.from_dict(rels_count_2, orient='index', columns=['count'])
df2['count'] = df2['count'] / df2['count'].sum()

df2['Model'] = 'Transformer_informative'

df = pd.concat([df, df2])

df = df.reset_index()
df = df.rename(columns={'index':'relation'})

df['relation'] = df['relation'].str.replace(' ', '\n')

sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})

ax = sns.barplot(x="relation", y="count", hue="Model", data=df)
# plot numbers on top of bars
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 0.01,
            '{:1.2f}'.format(height),
            ha="center")
ax.set_xlabel("Category")
ax.set_ylabel("Count")
plt.xticks()
plt.show()

In [ ]:
# plot pie chart of relations

import matplotlib.pyplot as plt
import numpy as np

counts, unique = rels_count_1.values(), rels_count_1.keys()
plt.pie(counts, labels=unique, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.show()

In [12]:
import h5py
import numpy as np
from tqdm import tqdm

test_path = "/home/maelic/Documents/SGDiff/datasets/vg/test.h5"

test_data = h5py.File(test_path, 'r')
print(test_data.keys()) 

generated_path = "/home/maelic/Documents/SGDiff/datasets/custom_pred_top5_intrinsic.h5"
custom_data = h5py.File(generated_path, 'r')

# get all image_ids from test_data not in custom_data
image_ids = []
for i, image_id in enumerate(tqdm(test_data['image_ids'])):
    if image_id not in custom_data['image_ids']:
        image_ids.append(i)

print(len(image_ids))

<KeysViewHDF5 ['attributes_per_object', 'image_ids', 'image_paths', 'object_attributes', 'object_boxes', 'object_ids', 'object_names', 'objects_per_image', 'relationship_ids', 'relationship_objects', 'relationship_predicates', 'relationship_subjects', 'relationships_per_image']>


  0%|          | 0/5096 [00:00<?, ?it/s]

100%|██████████| 5096/5096 [00:36<00:00, 140.81it/s]

103


In [13]:
print(image_ids)

[41, 97, 127, 155, 200, 207, 259, 305, 308, 313, 346, 349, 497, 557, 562, 615, 644, 683, 698, 813, 847, 895, 943, 1014, 1123, 1222, 1289, 1304, 1325, 1343, 1549, 1621, 1627, 1697, 1709, 1712, 1751, 1824, 2109, 2141, 2171, 2185, 2304, 2310, 2321, 2405, 2407, 2470, 2483, 2493, 2728, 2748, 2781, 2783, 2811, 2845, 2879, 2889, 2906, 2929, 2938, 2965, 2993, 3043, 3102, 3212, 3247, 3354, 3396, 3449, 3524, 3576, 3595, 3600, 3622, 3626, 3727, 3756, 3787, 3809, 3954, 4008, 4078, 4091, 4153, 4217, 4244, 4251, 4324, 4372, 4514, 4531, 4658, 4740, 4778, 4871, 4956, 4966, 4974, 4975, 4982, 5050, 5061]


In [16]:
source = "/home/maelic/Documents/SGDiff/test_results_baseline/scene_graph/"
import os

files = os.listdir(source)

# get files in order
files = sorted(files, key=lambda x: int(x.split('_')[0]))

for i in image_ids:
    print(files[i])
    # remove file
    os.remove(source + files[i])

41_graph.png
97_graph.png
127_graph.png
155_graph.png
200_graph.png
207_graph.png
259_graph.png
305_graph.png
308_graph.png
313_graph.png
346_graph.png
349_graph.png
497_graph.png
557_graph.png
562_graph.png
615_graph.png
644_graph.png
683_graph.png
698_graph.png
813_graph.png
847_graph.png
895_graph.png
943_graph.png
1014_graph.png
1123_graph.png
1222_graph.png
1289_graph.png
1304_graph.png
1325_graph.png
1343_graph.png
1549_graph.png
1621_graph.png
1627_graph.png
1697_graph.png
1709_graph.png
1712_graph.png
1751_graph.png
1824_graph.png
2109_graph.png
2141_graph.png
2171_graph.png
2185_graph.png
2304_graph.png
2310_graph.png
2321_graph.png
2405_graph.png
2407_graph.png
2470_graph.png
2483_graph.png
2493_graph.png
2728_graph.png
2748_graph.png
2781_graph.png
2783_graph.png
2811_graph.png
2845_graph.png
2879_graph.png
2889_graph.png
2906_graph.png
2929_graph.png
2938_graph.png
2965_graph.png
2993_graph.png
3043_graph.png
3102_graph.png
3212_graph.png
3247_graph.png
3354_graph.png
3396_

In [6]:
source = "/home/maelic/Documents/SGDiff/test_set_motifs_intrinsic_only/gt_img/"
import os

files = os.listdir(source)

# get files in order
files = sorted(files, key=lambda x: int(x.split('_')[0]))

for f in files[:1000]:
    # remove file
    os.remove(source + f)

In [3]:
# copy first 100 files from one folder to the other
source = '/home/maelic/Documents/SGDiff/test_set_motifs_intrinsic_only/img'
dest = '/home/maelic/Documents/SGDiff/comparison/to_test/'

import os
import shutil

files = os.listdir(source)

# get files in order
files = sorted(files, key=lambda x: int(x.split('_')[0]))

for f in files[:1000]:
    shutil.copy(source + '/' + f, dest + '/' + f)


In [29]:
source = '/home/maelic/Documents/SGDiff/test_set_motifs_intrinsic/temp/gt_img'
dest = '/home/maelic/Documents/SGDiff/test_set_motifs_intrinsic/temp/img_intrins'
dest = '/home/maelic/Documents/SGDiff/test_set_motifs_intrinsic/temp/img_baseline'


# compare names in both folders and verify it's the same
files1 = os.listdir(source)
files2 = os.listdir(dest)
files1.sort()
files2.sort()

# check length equal
print(len(files1))
print(len(files2))
assert len(files1) == len(files2)

for i in range(len(files1)):
    if files1[i] != files2[i]:
        print('error')
        break

1000
1000


In [9]:
import h5py
import numpy as np

# Load the data
dat_path = "/home/maelic/Documents/SGDiff/datasets/vg/test.h5"
dat = h5py.File(dat_path, 'r')

print(len(dat))

13


In [13]:
imgs = dat['image_paths']

with open("test.txt", "w") as f:
    for i in range(len(imgs)):
        # decode line in utf 8
        img = imgs[i].decode("utf-8")
        f.write(str(img) + "\n")

In [14]:
import shutil
import os

img_folder = "/home/maelic/Documents/Datasets/VG/VG_100K/"
dest_folder = "/home/maelic/Documents/Datasets/VG/VG_100K_test/"
# create destination folder
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
for img in imgs:
    img = img.decode("utf-8")
    img_path = img_folder + img
    # copy to destination
    dest_path = dest_folder + img
    shutil.copyfile(img_path, dest_path)

In [8]:
print(dat.keys())

for d in dat.keys():
    print(dat[d][59])

<KeysViewHDF5 ['attributes_per_object', 'image_ids', 'image_paths', 'object_attributes', 'object_boxes', 'object_ids', 'object_names', 'objects_per_image', 'relationship_ids', 'relationship_objects', 'relationship_predicates', 'relationship_subjects', 'relationships_per_image']>
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1]
b'2342709'
b'2342709.jpg'
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1]
[[ 39 122 401 223]
 [ 84  16 637 157]
 [530 152 989 317]
 [  0 189 985 576]
 [508 313 982 487]
 [ 77 105 500 325]
 [ 17 334 513 401]
 [408 376 961 444]
 [153 397 953 570]
 [ 14 118 879 266]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]
 [ -1  -1  -1 

In [3]:
print(len(dat['image_ids']))

62565
